# Crowd Counting Pipeline — Training on Colab

This notebook clones the repository and runs the training scripts on a GPU.
All logic lives in the Python package — this notebook just orchestrates.

**Requirements**: Colab with GPU runtime (Runtime > Change runtime type > GPU)


## 1. Setup: Clone Repo and Install


In [ ]:
# Clone the repository (replace with your GitHub URL)
!git clone https://github.com/YOUR_USERNAME/crowd-counting-pipeline.git
%cd crowd-counting-pipeline

# Install the package with training dependencies
%pip install -e ".[train]" -q


## 2. Download Dataset


In [ ]:
# Option A: Download from Kaggle (requires kaggle.json)
# Upload your kaggle.json first, then run:
# !mkdir -p ~/.kaggle && cp kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
# !python -m crowd_counting.data.download --output-dir ./data

# Option B: If you already have the dataset on Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Adjust path to where your dataset is on Drive
!ln -sf /content/drive/MyDrive/ShanghaiTech ./data/ShanghaiTech


## 3. Verify GPU


In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")


## 4. Train on ShanghaiTech Part B


In [ ]:
# Train with wandb logging (sign in when prompted)
!python scripts/train.py --config configs/shb.yaml --wandb --epochs 200


## 5. Train on ShanghaiTech Part A


In [ ]:
# Train on Part A (denser crowds, adaptive sigma)
!python scripts/train.py --config configs/sha.yaml --wandb --epochs 200


## 6. Evaluate


In [ ]:
# Evaluate on Part B test set
!python scripts/evaluate.py --config configs/shb.yaml --checkpoint weights/best_model.pth --save-viz

# Evaluate on Part A test set
!python scripts/evaluate.py --config configs/sha.yaml --checkpoint weights/best_model.pth --save-viz


## 7. Export to ONNX


In [ ]:
!python scripts/export_onnx.py --checkpoint weights/best_model.pth


## 8. Save Weights to Google Drive


In [ ]:
import shutil
import os

drive_output = '/content/drive/MyDrive/crowd-counting-weights'
os.makedirs(drive_output, exist_ok=True)

# Copy trained weights
for f in ['best_model.pth', 'checkpoint.pth', 'csrnet.onnx']:
    src = f'weights/{f}'
    if os.path.exists(src):
        shutil.copy2(src, drive_output)
        print(f'Saved {f} to Drive')

# Copy evaluation visualizations
if os.path.exists('outputs/evaluation_samples.png'):
    shutil.copy2('outputs/evaluation_samples.png', drive_output)
    print('Saved evaluation visualization to Drive')

print(f'\nAll outputs saved to: {drive_output}')
